In [7]:
#get document frequency for terms in the same article/instance; and get each sentence's BOW; and get each question's BOW;
#get each sentence's length; get each article's average length of sentences
import csv
import json
import copy
import os
import nltk
from math import log
from collections import defaultdict, Counter
from sklearn.feature_extraction import DictVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from scipy.spatial.distance import cosine as cos_distance
from sklearn.decomposition import TruncatedSVD
from nltk.tag import StanfordNERTagger
def get_BOW(text):
    BOW = {}
    for word in text:
        BOW[word] = BOW.get(word,0) + 1
    return BOW
def lemmatize(word):
    lemma = lemmatizer.lemmatize(word,'v')
    if lemma == word:
        lemma = lemmatizer.lemmatize(word,'n')
    return lemma
def input_data():
    base_path = os.path.join('')
    train_file = base_path + 'data/QA_train.json'
    train_data = json.load(open(train_file))
    test_file = base_path + 'data/QA_test.json'
    test_data = json.load(open(test_file))
    dev_file = base_path + 'data/QA_dev.json'
    dev_data = json.load(open(dev_file))

    return train_data,test_data,dev_data
def transform_text(text):
    text = nltk.word_tokenize(text)
    text = [lemmatize(word.lower()) for word in text]
    result = []
    for word in text:
        if word not in stopwords and word not in punctuations:
            result.append(word)
    return result
def get_Docfrequency_SentenceBOW(dataset):
    #save dics, each dictionary contains document frequencies for all terms in the same article
    question_list = []
    #save lists, each list represent an article, saving sentences' bow
    total_sentence_bow = []
    #save lists, each list represent an article, saving questions' bow
    total_question_bow = []
    #save lists, each list represent all sentences' lengthes.
    sent_lengthes = []
    #save a list, each item represents the average length of sentences
    avg_lengthes = []
    #
    answer_id = []
    
    for article in dataset:
        #Docfrequency
        article_dic = defaultdict(list)
        keyterms = [] #save all distinct terms in questions
        
        #SentenceBOW
        bow_list = []
        
        #QuestionBOW
        que_list = []
        
        #SentenceLength
        sent_len = []
        
        #TotalLength
        total_len = 0
        
        #RightAnser
        right_answer = []
        
        qas = article['qa']
        sentences = article['sentences']
        for qa in qas:
            question = qa['question']
            newquestion = transform_text(question)
            #QuestionBOW
            que_list.append(get_BOW(newquestion))
            answer = qa['answer_sentence']
            right_answer.append(answer)
            
            keyterms.extend(newquestion)
        keyterms = set(keyterms)
        
        #save sentences' BOW in list sen_BOW
        sen_words = []
        for sent in sentences:
            sent = transform_text(sent)
            #Docfrequency
            sen_words.append(sent)
            
            #SentenceBOW
            bow_list.append(get_BOW(sent))
            
            #SentenceLength
            sent_len.append(len(sent))
            
            #TotalLength
            total_len += len(sent)
            
        
        #calculate doc frequency    
        for term in keyterms:
            for i,bow in enumerate(sen_words):
                if term in bow:
                    article_dic[term].append(i)
                    
        #Docfrequency
        question_list.append(article_dic)
        #SentenceBOW
        total_sentence_bow.append(bow_list)
        #QuestionBOW
        total_question_bow.append(que_list)
        #SentenceLength
        sent_lengthes.append(sent_len)
        #AverageLength
        avg_lengthes.append(float(total_len)/len(sentences))
        #
        answer_id.append(right_answer)
        
    return question_list, total_sentence_bow, total_question_bow, sent_lengthes, avg_lengthes, answer_id

train,test,dev = input_data()
lemmatizer = nltk.stem.wordnet.WordNetLemmatizer()
stopwords = nltk.corpus.stopwords.words('english')
punctuations = [',','\'\'','?','\'','.','%','(',')',';','``']
question_list, total_sentence_bow, total_question_bow, sent_lengthes, avg_lengthes, answer_id_list= get_Docfrequency_SentenceBOW(train)

In [8]:
import math
def BM25(articles_index,k1,k2,b):
    total_queries = len(total_question_bow[articles_index])
    count = 0
    for index in range(len(total_question_bow[articles_index])):
        answer_id = answer_id_list[articles_index][index]
        guess_id = find_max_score_sentence(articles_index,index,k1,k2,b)
        if answer_id == guess_id:
            count += 1   
    accurancy = float(count)/total_queries  
    return accurancy

def find_max_score_sentence(articles_index,index,k1,k2,b):

    query_dict = total_question_bow[articles_index][index]
    max_score = 0
    guess_sentence = 0
    for index in range(len(total_sentence_bow[articles_index])):     
        score = 0  
        sentence_dict = total_sentence_bow[articles_index][index]
        for word in query_dict:
            document_fre_list = question_list[articles_index].get(word,None)
            
            
            N = len(total_sentence_bow[articles_index])
            n_qi = 0
            if document_fre_list != None:
                n_qi = len(document_fre_list)
            else:
                n_qi = 0
            fi = sentence_dict.get(word,0)
            qfi = query_dict.get(word,0)
            dl = sent_lengthes[articles_index][index]
            avgdl = avg_lengthes[articles_index]
            
            K = k1*(1-b+b*(float(dl)/avgdl)) 
            W = math.log((N-n_qi+0.5)/(n_qi+0.5))
            R = (fi*(k1+1))/(fi+K)*qfi*(k2+1)/(qfi+k2)
            score += W*R
        if score > max_score:
            max_score = score
            guess_sentence = index
    return guess_sentence

In [9]:
accurancy = 0.0

k1_list = [1.1]
k2_list = [110]
b_list = [0.18]

test_length = len(train)
for k1 in k1_list:
    for k2 in k2_list:
        for b in b_list:
            accurancy = 0.0
            for i in range(0,test_length):
                i_accurancy = BM25(i,k1,k2,b)
                accurancy += i_accurancy
            average_accurancy = accurancy/test_length
            print "Accurancy of BM25 with k1:",k1," k2:",k2," b:",b
            print average_accurancy



KeyboardInterrupt: 

In [1]:
#Accurancy of BM25 with k1: 1.1  k2: 110  b: 0.18
#0.683486756477
print 999,'a'

999 a


In [13]:

article = train[0]
qas = article['qa']
sentences = article['sentences']

question_types = ['what','where','which','whom','how','when','why']
for i in range(len(qas)):
    #print sentences[qas[i]['answer_sentence']]
    #print nltk.word_tokenize(sentences[qas[i]['answer_sentence']])
    #print transform_text(sentences[qas[i]['answer_sentence']])
    print 'Question:',qas[i]['question']
    #print 'Answer:',qas[i]['answer']
    #NER = st.tag(transform_text(sentences[qas[0]['answer_sentence']]))




Question: What does LP stand for when it comes to time capacity?
Question: What are common diameters found in phonograph records?
Question: What is a gramophone or 'vinyl' record?
Question: Where does the groove on a vinyl record typically start?
Question: In what method is the rotational speed measured in?
Question: What was the primary use of a phonographic disc record?
Question: Which year did vinyl records leave the main steam media market?
Question: From the 1990s to 2010s who was the primary consumer of vinyl records?
Question: Approximately how many phonograph records were sold in 2014?
Question: What is the niche market of phonograph record fans known as?
Question: What was the original intent of the phonautograph?
Question: In what years where phonautograms converted to audible sound?
Question: What year were the earliest known recordings of sound?
Question: By whom was the phonautograms patented?
Question: In what year was phonautograms patented?
Question: In what year did Th

In [5]:
#display some qa
test = []
for qu in qas[:5]:
    print 'Q',qu['question']
    print 'A',qu['answer']
    test.append(sentences[qu['answer_sentence']])
    #print '--',test

Q What does LP stand for when it comes to time capacity?
A long playing
Q What are common diameters found in phonograph records?
A 12", 10", 7"
Q What is a gramophone or 'vinyl' record?
A analogue sound storage medium
Q Where does the groove on a vinyl record typically start?
A near the periphery
Q In what method is the rotational speed measured in?
A rpm


In [116]:
# -*- coding: utf-8 -*-
#get from stackoverflow
from nltk import ne_chunk, pos_tag, word_tokenize
from nltk.tree import Tree
import sys 


def get_continuous_chunks(text,texttype):
    text = text.encode('ascii','replace')
    pos =  pos_tag(word_tokenize(text))
    if texttype==0:
        print pos
        grammar = r"""
                    NP: 
                        {<DT>?<JJ.*>*<NN.*>+}
                        {<DT>?<JJ.*>*<VBG.*>+}
                        {<NNP>+}
                    VB:
                        {<IN><NP>}


                    """
    elif texttype==1:
        if pos[0]==('Which', 'JJ'):
            pos[0]=('Which', 'WHICH')
        print pos
        grammar = r"""
                    WH: 
                        {<W.*><NP>+}
                        {<WRB><JJ>+}
                        {<WHICH><NN>}
                        {<W.*>}
                    NP: 
                        {<DT>?<JJ.*>*<NN.*>+}
                        {<DT>?<JJ.*>*<VBG.*>+}
                        {<NNP>+}
                    """
    elif texttype==2:
        if pos[0]==('Which', 'JJ'):
            pos[0] = ('Which', 'WHICH')
        for i,item in enumerate(pos):   
            #print ('item',item)
            word = item[0].lower()
            ppos = item[1]
            if word=='what' and ppos=='WP':
                pos[i] = ('what', 'WHAT')   
            elif word=='which' and ppos=='JJ':
                pos[i] = ('which', 'WHICH')  
        print pos
        grammar = r"""
                    V: {<V.*>}          # Verb
                    XX1:
                        {<IN>?<WHAT><V>?<DT>?<JJ.*|CD>*<V>?<IN>?<NN.*>+}
                        {<IN>?<WHAT><V>?<DT>?<JJ.*|CD>*<V>?<IN>?<VBG.*>+}
                        }<IN>?<WHAT><V>?<DT>?<JJ.*|CD>*<V>?<IN>?{  
                    """
    cp = nltk.RegexpParser(grammar) 
    result = []
    tree = cp.parse(pos)
    for subtree in tree.subtrees():
        if texttype==0 and subtree.label() != 'S':
            phrase = u' '.join([word for word,pos in subtree.leaves()])
            result.append((subtree.label(),phrase))
            print subtree.label(),phrase
        elif texttype==1 and subtree.label() != 'S':
            phrase = u' '.join([word for word,pos in subtree.leaves()])
            result.append((subtree.label(),phrase))
            print subtree.label(),phrase
        elif texttype==2 and subtree.label()=='XX1':
            phrase = u' '.join([word for word,pos in subtree.leaves()])
            result.append((subtree.label(),phrase))
            phrase_list = phrase.split()
            if len(phrase_list)>1:
                ph = phrase_list[-1]
            else:
                ph = phrase
            print subtree.label()+':'+phrase+'->'+ph
            
    return result

In [117]:
#Test the queries
for qu in qas[100:200]:
    text = qu['question']
    if 'what' in text.lower():
        print text
        get_continuous_chunks(text,2)
        print '-----'
    else:
        print 'jump'

What does LP stand for when it comes to time capacity?
[('what', 'WHAT'), ('does', 'VBZ'), ('LP', 'NNP'), ('stand', 'VB'), ('for', 'IN'), ('when', 'WRB'), ('it', 'PRP'), ('comes', 'VBZ'), ('to', 'TO'), ('time', 'NN'), ('capacity', 'NN'), ('?', '.')]
XX1:LP->LP
-----
What are common diameters found in phonograph records?
[('what', 'WHAT'), ('are', 'VBP'), ('common', 'JJ'), ('diameters', 'NNS'), ('found', 'VBN'), ('in', 'IN'), ('phonograph', 'NN'), ('records', 'NNS'), ('?', '.')]
XX1:diameters->diameters
-----
What is a gramophone or 'vinyl' record?
[('what', 'WHAT'), ('is', 'VBZ'), ('a', 'DT'), ('gramophone', 'NN'), ('or', 'CC'), ("'vinyl", 'POS'), ("'", "''"), ('record', 'NN'), ('?', '.')]
XX1:gramophone->gramophone
-----
jump
In what method is the rotational speed measured in?
[('In', 'IN'), ('what', 'WHAT'), ('method', 'NN'), ('is', 'VBZ'), ('the', 'DT'), ('rotational', 'JJ'), ('speed', 'NN'), ('measured', 'VBN'), ('in', 'IN'), ('?', '.')]
XX1:method->method
-----
What was the primar

In [35]:
#Test the answer_sentence
for sen in test:
    print sen
    get_continuous_chunks(sen,0)

{u'qa': [{u'question': u'What year did the Crimean War begin?', u'id': 1}, {u'question': u'What year did the Crimean War end?', u'id': 2}, {u'question': u'Who controlled the rights of Christian minorities in the Holy Land?', u'id': 3}, {u'question': u'Who promoted the rights of the Catholic people?', u'id': 4}, {u'question': u'Who promoted the rights of the Eastern Orthodox Christians?', u'id': 5}, {u'question': u'Which two figures refused to back down after the churches worked out their disagreements?', u'id': 6}, {u'question': u'Who wanted the Orthodox subjects to be placed under their protection?', u'id': 7}, {u'question': u'Who arranged the compromise that Nicholas agreed to?', u'id': 8}, {u'question': u'Who demanded the changes that Nicholas later refused?', u'id': 9}, {u'question': u'What month did the Ottomans declare war on Russia?', u'id': 10}, {u'question': u'Where did the war begin?', u'id': 11}, {u'question': u'Russian troops took over which provinces first?', u'id': 12}, {

AttributeError: 'dict' object has no attribute 'encode'

In [10]:
#display some qa
test = []
for qu in qas[:100]:
    print '---'
    text = qu['question']
    print text
    text = text.encode('ascii','replace')
    pos =  pos_tag(word_tokenize(text))
    print pos

    

---
What does LP stand for when it comes to time capacity?
[('What', 'WP'), ('does', 'VBZ'), ('LP', 'NNP'), ('stand', 'VB'), ('for', 'IN'), ('when', 'WRB'), ('it', 'PRP'), ('comes', 'VBZ'), ('to', 'TO'), ('time', 'NN'), ('capacity', 'NN'), ('?', '.')]
---
What are common diameters found in phonograph records?
[('What', 'WP'), ('are', 'VBP'), ('common', 'JJ'), ('diameters', 'NNS'), ('found', 'VBN'), ('in', 'IN'), ('phonograph', 'NN'), ('records', 'NNS'), ('?', '.')]
---
What is a gramophone or 'vinyl' record?
[('What', 'WP'), ('is', 'VBZ'), ('a', 'DT'), ('gramophone', 'NN'), ('or', 'CC'), ("'vinyl", 'POS'), ("'", "''"), ('record', 'NN'), ('?', '.')]
---
Where does the groove on a vinyl record typically start?
[('Where', 'WRB'), ('does', 'VBZ'), ('the', 'DT'), ('groove', 'NN'), ('on', 'IN'), ('a', 'DT'), ('vinyl', 'NN'), ('record', 'NN'), ('typically', 'RB'), ('start', 'VB'), ('?', '.')]
---
In what method is the rotational speed measured in?
[('In', 'IN'), ('what', 'WP'), ('method', 'NN

In [37]:
for sent in test:
    print sent
    sent = sent.encode('ascii','replace')
    pos =  pos_tag(word_tokenize(sent))
    print(nltk.ne_chunk(pos, binary=False))

Phonograph records are generally described by their diameter in inches (12", 10", 7"), the rotational speed in rpm at which they are played (16 2⁄3, 33 1⁄3, 45, 78), and their time capacity resulting from a combination of those parameters (LP – long playing 33 1⁄3 rpm, SP – 78 rpm single, EP – 12-inch single or extended play, 33 or 45 rpm); their reproductive quality or level of fidelity (high-fidelity, orthophonic, full-range, etc.), and the number of audio channels provided (mono, stereo, quad, etc.).
(S
  (GPE Phonograph/NN)
  records/NNS
  are/VBP
  generally/RB
  described/VBN
  by/IN
  their/PRP$
  diameter/NN
  in/IN
  inches/NNS
  (/(
  12/CD
  ''/''
  ,/,
  10/CD
  ''/''
  ,/,
  7/CD
  ''/''
  )/)
  ,/,
  the/DT
  rotational/JJ
  speed/NN
  in/IN
  rpm/NN
  at/IN
  which/WDT
  they/PRP
  are/VBP
  played/VBN
  (/(
  16/CD
  2/CD
  ?/.
  3/CD
  ,/,
  33/CD
  1/CD
  ?/.
  3/CD
  ,/,
  45/CD
  ,/,
  78/CD
  )/)
  ,/,
  and/CC
  their/PRP$
  time/NN
  capacity/NN
  resulting/VBG
 

In [136]:
# -*- coding: utf-8 -*-
#get from stackoverflow
from nltk import ne_chunk, pos_tag, word_tokenize
from nltk.tree import Tree
import sys 

#Only consider queries with 'What'/'what'
def getHeadWord(text):
    text = text.encode('ascii','replace')
    pos =  pos_tag(word_tokenize(text))
    if pos[0]==('Which', 'JJ'):
        pos[0] = ('Which', 'WHICH')
    for i,item in enumerate(pos):   
        #print ('item',item)
        word = item[0].lower()
        ppos = item[1]
        if word=='what' and (ppos=='WP' or ppos=='WDT'):
            pos[i] = ('what', 'WHAT')   
        elif word=='which' and ppos=='JJ':
            pos[i] = ('which', 'WHICH')  
    print pos
    grammar = r"""
                V: {<V.*>}          # Verb
                HEAD:
                    {<IN>?<WHAT><V>?<DT>?<JJ.*|CD>*<V>?<IN>?<NN.*>+}
                    {<IN>?<WHAT><V>?<DT>?<JJ.*|CD>*<V>?<IN>?<VBG.*>+}
                    }<IN>?<WHAT><V>?<DT>?<JJ.*|CD>*<V>?<IN>?{  
                """
    cp = nltk.RegexpParser(grammar) 
    result = []
    tree = cp.parse(pos)
    for subtree in tree.subtrees():
        if subtree.label()=='HEAD':
            phrase = u' '.join([word for word,pos in subtree.leaves()])
            phrase_list = phrase.split()
            if len(phrase_list)>1:
                ph = phrase_list[-1]
            else:
                ph = phrase
            result.append(ph)
            print subtree.label()+':'+phrase+'->'+ph       
    return result


In [137]:
#Test the queries
headwords = []
for qu in qas[100:200]:
    text = qu['question']
    if 'what' in text.lower():
        print 'Q:',text
        print 'A:',qu['answer']
        headwords += getHeadWord(text)
        print '-----'
print (len(headwords))    

Q: What record company released recordings of 'Buena' and 'Tuff Enuff' in 1980?
A: Stiff Records
[('what', 'WHAT'), ('record', 'NN'), ('company', 'NN'), ('released', 'VBD'), ('recordings', 'NNS'), ('of', 'IN'), ("'Buena", 'NNP'), ("'", 'POS'), ('and', 'CC'), ("'Tuff", 'POS'), ('Enuff', 'NNP'), ("'", "''"), ('in', 'IN'), ('1980', 'CD'), ('?', '.')]
HEAD:record company->company
-----
Q: What was the result of the use of this release of 78 rpm vinyl when used in vintage jukeboxes?
A: they were destroyed
[('what', 'WHAT'), ('was', 'VBD'), ('the', 'DT'), ('result', 'NN'), ('of', 'IN'), ('the', 'DT'), ('use', 'NN'), ('of', 'IN'), ('this', 'DT'), ('release', 'NN'), ('of', 'IN'), ('78', 'CD'), ('rpm', 'NNS'), ('vinyl', 'NN'), ('when', 'WRB'), ('used', 'VBN'), ('in', 'IN'), ('vintage', 'NN'), ('jukeboxes', 'NNS'), ('?', '.')]
HEAD:result->result
-----
Q: What was needed to successfully play the newer released 79 rpm vinyl?
A: lightweight tone arms and jewel needles
[('what', 'WHAT'), ('was', 'V

In [123]:
a =[]
b=[]
print a
print a+b

[]
[]
